<a href="https://colab.research.google.com/github/Jaisman/GDSC-FOSS/blob/main/spotify_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('spotify_millsongdata.csv', engine='python', on_bad_lines='skip')

In [ ]:
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
data.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [ ]:
data.drop('link', axis=1, inplace=True)

In [ ]:
data.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [ ]:
data['text'] = data['text'].str.lower().replace(r'^a-zA-Z0-9','').replace(r'\n', ' ', regex=True)

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def token(text):
    text = nltk.word_tokenize(text)
    a = [stemmer.stem(w) for w in text]
    return " ".join(a)

In [ ]:
data['text'].apply(lambda x: token(x))

,text
0,"look at her face , it 's a wonder face and it ..."
1,"take it easi with me , pleas touch me gentli l..."
2,i 'll never know whi i had to go whi i had to ...
3,make somebodi happi is a question of give and ...
4,make somebodi happi is a question of give and ...
...,...
3315,lord i do n't know which way i am go which way...
3316,"so it 's over , thi time i know it 's gone sal..."
3317,"the bird they sang , at break of day start aga..."
3318,"oh morn come burst , the cloud amen lift off t..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
tfidf_matrix = tfid.fit_transform(data['text'])

In [ ]:
similar = cosine_similarity(tfidf_matrix)

In [ ]:
def recommender(song_name):
  print(f"Searching for: {song_name}")
  song_name = song_name.lower()
  matches = data[data['song'].str.lower() == song_name]

  if not matches.empty:
    idx = matches.index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
      song.append(data.iloc[s_id[0]].song)
    return song
  else:
    return "Song not found"

In [ ]:
import pickle

In [ ]:
recommender("Bang")

Searching for: Bang
Found 2 matches


['The Prime Of Your Love',
 'Bang-A-Boomerang',
 'Boom Boom Boom',
 "Boom, Boom, Ain't It Great To Be Crazy?"]

In [ ]:
pickle.dump(similar, open('similar.pkl', 'wb'))

In [ ]:
pickle.dump(data, open('data.pkl', 'wb'))